# Download Multi-Platform Online Courses Dataset from Kaggle

In [40]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("everydaycodings/multi-platform-online-courses-dataset")

print("Path to dataset files:", path)

Path to dataset files: C:\Users\flees\.cache\kagglehub\datasets\everydaycodings\multi-platform-online-courses-dataset\versions\2


## Functions to clean kaggle datasets

In [41]:
import pandas as pd
import numpy as np

target_columns = [
    'url', 'title', 'author', 'students', 'rating',
    'difficulty', 'skills', 'description', 'price', 'source'
]

coursera_rename_map = {
    'course': 'title',
    'reviewcount': 'students',
    'level': 'difficulty'
}

coursera_kaggle_df = pd.read_csv("multi-platform-online-courses-dataset/Coursera.csv")
print(f"Columns before preprocessing: \n{coursera_kaggle_df.columns}\nSize: {len(coursera_kaggle_df)}\n")

# rename columns using map
coursera_kaggle_df = coursera_kaggle_df.rename(columns=coursera_rename_map)
# delete non-target columns
coursera_kaggle_df = coursera_kaggle_df[[col for col in coursera_kaggle_df.columns if col in target_columns]]

for col in target_columns:
    if col not in coursera_kaggle_df.columns:
        coursera_kaggle_df[col] = np.nan

coursera_kaggle_df = coursera_kaggle_df[target_columns]

coursera_kaggle_df = coursera_kaggle_df[~coursera_kaggle_df['skills'].isna()]
coursera_kaggle_df = coursera_kaggle_df[coursera_kaggle_df['skills'].astype(str).str.strip() != ""]
coursera_kaggle_df = coursera_kaggle_df[coursera_kaggle_df['skills'].astype(str) != "[]"]

print(f"Columns after preprocessing: \n{coursera_kaggle_df.columns}\nSize: {len(coursera_kaggle_df)}")

Columns before preprocessing: 
Index(['partner', 'course', 'skills', 'rating', 'reviewcount', 'level',
       'certificatetype', 'duration', 'crediteligibility'],
      dtype='object')
Size: 1139

Columns after preprocessing: 
Index(['url', 'title', 'author', 'students', 'rating', 'difficulty', 'skills',
       'description', 'price', 'source'],
      dtype='object')
Size: 1088


## Clean edx data set

In [42]:
edx_kaggle_df = pd.read_csv("multi-platform-online-courses-dataset/edx.csv")
print(f"Columns before preprocessing: \n{edx_kaggle_df.columns}\nSize: {len(edx_kaggle_df)}\n")

edx_rename_map = {
    'link': 'url',
    'institution': 'author',
    'level': 'difficulty',
    'associatedskills': 'skills'
}

# rename columns using map
edx_kaggle_df = edx_kaggle_df.rename(columns=edx_rename_map)

edx_kaggle_df = edx_kaggle_df[edx_kaggle_df['language'] == "English"]

valid_subjects = ["Data Analysis & Statistics", "Computer Science"]
edx_kaggle_df = edx_kaggle_df[edx_kaggle_df["subject"].isin(valid_subjects)]

# delete non-target columns
edx_kaggle_df = edx_kaggle_df[[col for col in edx_kaggle_df.columns if col in target_columns]]

# add missing target columns
for col in target_columns:
    if col not in edx_kaggle_df.columns:
        edx_kaggle_df[col] = np.nan


edx_kaggle_df["source"] = "edx"

edx_kaggle_df = edx_kaggle_df[target_columns]

edx_kaggle_df = edx_kaggle_df[~edx_kaggle_df['skills'].isna()]
edx_kaggle_df = edx_kaggle_df[edx_kaggle_df['skills'].astype(str).str.strip() != ""]
edx_kaggle_df = edx_kaggle_df[edx_kaggle_df['skills'].astype(str) != "[]"]


print(f"Columns after preprocessing: \n{edx_kaggle_df.columns}\nSize: {len(edx_kaggle_df)}")


Columns before preprocessing: 
Index(['title', 'link', 'institution', 'subject', 'level', 'prerequisites',
       'language', 'videotranscript', 'associatedprograms',
       'associatedskills'],
      dtype='object')
Size: 816

Columns after preprocessing: 
Index(['url', 'title', 'author', 'students', 'rating', 'difficulty', 'skills',
       'description', 'price', 'source'],
      dtype='object')
Size: 210


## Move files into ml directory & check content

 We will not use skillshare.csv and Udemy.csv dataset due to lack of skill columns.



In [43]:
import os

path = "multi-platform-online-courses-dataset"
datasets = []
df = pd.DataFrame()

for filename in os.listdir(path):
    if filename.endswith(".json"):
        dataset = pd.read_json(os.path.join(path, filename))
        datasets.append(dataset)
        df = pd.concat([df, dataset], ignore_index=True)
        print(filename)
        print(f"{dataset.columns}\n\n")

df = pd.concat([df, edx_kaggle_df], ignore_index=True)

df.to_csv("courses.csv", index=False)

coursera_clear.json
Index(['url', 'title', 'author', 'students', 'rating', 'difficulty', 'skills',
       'description', 'price', 'source'],
      dtype='object')


stepik_clear.json
Index(['url', 'title', 'author', 'students', 'rating', 'difficulty', 'skills',
       'description', 'price', 'source'],
      dtype='object')




## Data normalization

#### Skills normalization
Google Cloud, Cloud Computing, Network Security, Management, Cloud Automation ----> ['Google Cloud', 'Cloud Computing', 'Network Security', 'Management', 'Cloud Automation']

In [44]:
def normalize_skills(skills):
    if isinstance(skills, str):
        return [skill for skill in skills.split(", ")]
    elif isinstance(skills, list):
        return skills

df["skills"] = df["skills"].apply(normalize_skills)
df.to_csv("courses1.csv", index=False)

## Generate pseudo-description for missing values

In [45]:
import requests

descriptions = []
df.to_csv("courses2.csv", index=False)

def generate_pseudo_description(course_line):
    skills_str = ", ".join(course_line["skills"])
    title = course_line.get("title", "Untitled course")
    print(f"\nTitle: {title}\nSkills: {skills_str}\n ")
    prompt = (
        f"Write a concise and engaging course description for an online educational course titled '{title}'. "
        f"The course covers the following skills: {skills_str}. "
        f"The description should be clear, informative, and suitable for potential learners. "
        f"Keep the tone professional yet accessible. The description should be around 3 to 7 sentences long. "
        f"Only return the course description. Do not include any additional explanations or formatting."
    )
    try:
        response = requests.post(
            "http://localhost:11434/api/generate",
            json={
                "model": "llama3",
                "prompt": prompt,
                "stream": False
            }
        )
        response.raise_for_status()
        print(response.json()["response"])
        descriptions.append(response.json()["response"])
        return response.json()["response"].strip()

    except Exception as e:
        print(f"Error generating description: {e}")
        return f"This course will teach you: {skills_str}. It is titled '{title}'."


def build_pseudo_description(course_line):
    skills_str = str(course_line["skills"])
    return f"This course will teach you: {skills_str}. It is titled '{course_line['title']}'."

# df['description'] = df['description'].fillna(df.apply(generate_pseudo_description, axis=1))
# df['description'] = df['description'].fillna(df.apply(generate_pseudo_description, axis=1))

mask = df['description'].isna() | df['description'].astype(str).str.strip().eq("")

df.loc[mask, 'description'] = df.loc[mask].apply(generate_pseudo_description, axis=1)


Title: CS50's Introduction toComputer Science
Skills: Resource Management, JavaScript (Programming Language), Forensic Sciences, Cryptography, SQL (Programming Language), Finance, Algorithms, Computer Science, HyperText Markup Language (HTML), Data Structures, Python (Programming Language), C (Programming Language), Security Software, Cascading Style Sheets (CSS)
 
Discover the foundations of computer science with CS50's Introduction to Computer Science! This comprehensive online course introduces you to a wide range of essential skills, from programming languages like JavaScript and Python, to data structures and algorithms. Explore topics such as resource management, cryptography, and SQL, while also delving into the world of finance and security software. As you learn, you'll gain hands-on experience with HTML, CSS, and C, plus a deeper understanding of computer science concepts and their applications in modern society. By the end of this course, you'll be equipped to tackle comple

In [50]:
# final version of courses with generated descriptions
df.to_csv("courses_final.csv", index=False)

# Create multi-vector

1. Add vector representation columns for each course:
    ```
   {
        "title_vector":[ ],
        "desc_vector": [ ],
        "skills_vector": [ ]
    }
   ```


In [55]:
from sentence_transformers import SentenceTransformer

model = SentenceTransformer('sentence-transformers/all-mpnet-base-v2')

def vectorize(row):
    return {
        "title_vector": model.encode(row["title"]),
        "desc_vector": model.encode(row.get("description", "")),
        "skills_vector": model.encode(", ".join(row["skills"]))
    }

# delete courses with NaN titles
df = df.dropna(subset=["title", "description", "skills"]).reset_index(drop=True)
df["vectors"] = df.apply(vectorize, axis=1)
print(df.head())

                                                 url  \
0  https://www.coursera.org/learn/python-for-appl...   
1  https://www.coursera.org/learn/python-crash-co...   
2    https://www.coursera.org/specializations/python   
3  https://www.coursera.org/learn/programming-in-...   
4  https://www.coursera.org/professional-certific...   

                                               title  \
0          Python for Data Science, AI & Development   
1                             Crash Course on Python   
2                Python for Everybody Specialization   
3                              Programming in Python   
4  Microsoft Python Development  Professional Cer...   

                       author   students  rating difficulty  \
0        Joseph Santarcangelo  1222909.0     4.6   Beginner   
1  Google Career Certificates  1256131.0     4.8   Beginner   
2   Charles Russell Severance  1797838.0     4.8   Beginner   
3        Taught by Meta Staff   124567.0     4.6   Beginner   
4          

In [ ]:
# !docker run -d -p 6333:6333 -p 6334:6334 -v C:\qdrant_data:/qdrant/storage --name qd qdrant/qdrant

In [ ]:
# w_title = 0.3
# w_desc = 0.1
# w_skills = 0.6
#
# points = []
#
# for idx, row in df.iterrows():
#     v_title = np.array(row["vectors"]["title_vector"])
#     v_desc = np.array(row["vectors"]["desc_vector"])
#     v_skills = np.array(row["vectors"]["skills_vector"])
#
#     # Нормализуем каждый вектор (по желанию)
#     v_title = v_title / np.linalg.norm(v_title)
#     v_desc = v_desc / np.linalg.norm(v_desc)
#     v_skills = v_skills / np.linalg.norm(v_skills)
#
#     # Взвешенное объединение
#     combined_vector = (
#         w_title * v_title +
#         w_desc * v_desc +
#         w_skills * v_skills
#     )
#
#     # Нормализация итогового вектора (по желанию, для косинусной метрики)
#     combined_vector = combined_vector / np.linalg.norm(combined_vector)
#
#     point = PointStruct(
#         id=idx,
#         vector=combined_vector.tolist(),
#         payload={
#             "title": row["title"],
#             "description": row["description"],
#             "skills": row["skills"]
#         }
#     )
#     points.append(point)
#
# # Отправляем в Qdrant
# client.upsert(collection_name="job_vectors", points=points)

In [ ]:
# import uuid
# import pandas as pd
# from sentence_transformers import SentenceTransformer
# from qdrant_client.models import PointStruct
#
#
# def prepare_qdrant_points(df: pd.DataFrame) -> list[PointStruct]:
#     points = []
#
#     for _, row in df.iterrows():
#         title = row["title"]
#         description = row["description"]
#         skills = row["skills"]
#
#         # Генерируем векторы
#         vec_title = model.encode(title).tolist()
#         vec_description = model.encode(description).tolist()
#         vec_skills = model.encode(", ".join(skills)).tolist()
#
#         # Строим структуру для Qdrant
#         point = PointStruct(
#             id=str(uuid.uuid4()),
#             payload={
#                 "title": title,
#                 "description": description,
#                 "skills": skills,
#                 "weights": {
#                     "title": 0.5,
#                     "description": 0.3,
#                     "skills": 0.2
#                 }
#             },
#             vectors={
#                 "title": vec_title,
#                 "description": vec_description,
#                 "skills": vec_skills
#             }
#         )
#         points.append(point)
#
#     return points

In [48]:
# from sentence_transformers import SentenceTransformer
#
# model = SentenceTransformer('all-MiniLM-L6-v2')
# #
# # course_texts = [course_to_text(course) for course in df]
# # embeddings = model.encode(course_texts, show_progress_bar=True)
